# Preparing the RASC Double Stars catalog for import into PiFinder

## Setup

In [35]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## importing the supplement

In [36]:
df = pd.read_excel (r'./RASC DS supplement 210710-import.xlsx')
df.columns = df.columns.str.strip()
df.drop(['Unnamed: 10', 'Unnamed: 11'], axis=1, inplace=True)
# drop repeated headers
rows_to_drop = df[df.apply(lambda x: (x == df.columns).all(), axis=1)].index
# Drop the identified rows
df.drop(index=rows_to_drop, inplace=True)
print(df.describe())
df.head()

        Target  Parent  WDS Pair   PA  Sep     M1   M2  Con  Notes     Type
count      129     119  244  221  244  244  244.0  244  112    244      244
unique     122     111  147   21  181  217   57.0   74   47    234        7
top     Target  Parent  WDS   AB   PA  Sep    5.7   M2  Com  Notes  Optical
freq         8       9    9   80    9    9   11.0    9    5      9       97


,Target,Parent,WDS,Pair,PA,Sep,M1,M2,Con,Notes,Type
0,WINTER (part 1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HD 21700,STFA 7,STFA 7,AB,234,44.1,7.4,7.8,Tau,Wide pair. Nearly equal. Oriented NE to SW. Yellow or orange and blue. Beside double HR 1065.,Physical
2,Phi Tau,SHJ 40,SHJ 40,AB,259,48.7,5.1,7.5,Tau,"Widely separated. Intense colours, gold and blue. Unrelated bright orange-red star opposite B.",Optical
3,NaN,NaN,WAL 29,AC,25,118.1,5.1,12.3,NaN,"Very dim partner, NNE.",Uncertain
4,32 Eri,STF 470,STF 470,AB,349,6.9,4.8,5.9,Eri,"Colourful pair, yellow and pale blue. Tight. May be called HR 1211 as opposed to HR 1212.",Uncertain


In [37]:
df['season'] = None

# Variable to hold the current season value
current_season = None
seasons = {
    "WINTER (part 1)": "Win1",
    "WINTER (part 2)": "Win2",
    "SPRING (part 1)": "Spr1",
    "SPRING (part 2)": "Spr2",
    "SPRING (part 3)": "Spr3",
    "SUMMER (part 1)": "Sum1",
    "SUMMER (part 2)": "Sum2",
    "SUMMER (part 3)": "Sum3",
    "AUTUMN (part 1)": "Aut1",
    "AUTUMN (part 2)": "Aut2",
}

# Iterate through the DataFrame
for index, row in df.iterrows():
    # Check if the 'Type' column is not NaN
    if pd.isna(row['Type']) and not pd.isna(row['Target']):
        # Update the current season using the value in the 'Target' column
        current_season = row['Target']
        #print(f"Updating to {current_season=} because of {row=}")
    else:
        # Set the current season for the row
        #print(current_season)
        df.at[index, 'season'] = seasons[current_season] if current_season is not None else nan
# Remove rows where 'Type' is NaN
df = df.dropna(subset=['Type'])
df['Pair'].fillna('AB', inplace=True)
df = df.loc[df['Target'] != 'Target']

In [38]:
supplement_df = df
supplement_df


,Target,Parent,WDS,Pair,PA,Sep,M1,M2,Con,Notes,Type,season
1,HD 21700,STFA 7,STFA 7,AB,234,44.1,7.4,7.8,Tau,Wide pair. Nearly equal. Oriented NE to SW. Yellow or orange and blue. Beside double HR 1065.,Physical,Win1
2,Phi Tau,SHJ 40,SHJ 40,AB,259,48.7,5.1,7.5,Tau,"Widely separated. Intense colours, gold and blue. Unrelated bright orange-red star opposite B.",Optical,Win1
3,NaN,NaN,WAL 29,AC,25,118.1,5.1,12.3,NaN,"Very dim partner, NNE.",Uncertain,Win1
4,32 Eri,STF 470,STF 470,AB,349,6.9,4.8,5.9,Eri,"Colourful pair, yellow and pale blue. Tight. May be called HR 1211 as opposed to HR 1212.",Uncertain,Win1
5,NaN,NaN,STF 470,AC,5,164.7,4.8,10.5,NaN,"Dim star beyond secondary, further north. Colourless.",Optical,Win1
6,Keid,STF 518,STF 518,AB,102,83.7,4.4,9.3,Eri,Very wide. Primary is orange or yellow. B is blue or red? B nearly due east.,Physical,Win1
7,NaN,NaN,STF 518,AC,98,78.1,4.4,11.2,NaN,"Another dim companion. C is very close to B, 9"" apart! C north of B.",Physical,Win1
8,NaN,NaN,STF 518,AD,38,481.4,4.4,12.6,NaN,Very faint and a long ways away. Looks like a field star. To the NE.,Optical,Win1
9,1 Cam,STF 550,STF 550,AB,309,10.4,5.8,6.8,Cam,A and B bright. B NW of A. Very tight. Both look blue-white. A showcase pair for many.,Uncertain,Win1
10,NaN,NaN,STF 550,BC,217,150.9,6.8,11.4,NaN,C much dimmer and well away to the SW. Looks pale blue?,Optical,Win1


## importing the main catalog

In [39]:
df = pd.read_excel (r'./RASC DS main checklist 210710.xlsx', skiprows=16)
df.columns = df.columns.str.strip()
# fix mismatch in name
df['Target'] = df['Target'].replace('The Trapezium', 'Trapezium')
df['Target'] = df['Target'].replace('U  Cyg', 'U Cyg')
df.drop(['Seen?'], axis=1, inplace=True)
# drop repeated headers
rows_to_drop = df[df.apply(lambda x: (x == df.columns).all(), axis=1)].index
# Drop the identified rows
df.drop(index=rows_to_drop, inplace=True)
df


,Target,Alternate ID,SAO,HIP,WDS,Con,MagC,RA 2000,Dec 2000,Mm,X,PSA
0,WINTER (part 1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HD 21700,BD+27 514,SAO 75964,HIP 16386,STFA 7,Tau,6.8,03 31.1,+27 44,100,45,15
2,Phi Tau,"52 Tau, Alkalbain I",SAO 76558,HIP 20250,SHJ 40,Tau,5,04 20.4,+27 21,90,45,15
3,32 Eri,HR 1212,SAO 130806,HIP 18255,STF 470,Eri,4.5,03 54.3,-02 57,90,140,17
4,Keid,Omicron 2 Eri,SAO 131063,HIP 19849,STF 518,Eri,4.4,04 15.2,-07 40,100,25,17
5,1 Cam,DL Cam,SAO 24672,HIP 21148,STF 550,Cam,5.4,04 32.0,+53 55,90,70,12
6,Beta Cam,10 Cam,SAO 13351,HIP 23522,S 459,Cam,4,05 03.4,+60 27,100,60,11
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,WINTER (part 2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Rho Ori,17 Ori,SAO 112528,HIP 24331,STF 654,Ori,4.6,05 13.3,+02 52,150,100,14


In [40]:
df.describe()

,Target,Alternate ID,SAO,HIP,WDS,Con,MagC,RA 2000,Dec 2000,Mm,X,PSA
count,117,110,110,109,110,110,110.0,110,110,110,110,110
unique,117,110,110,109,110,46,47.0,110,108,5,11,47
top,WINTER (part 1),BD+27 514,SAO 75964,HIP 16386,STFA 7,Com,5.2,03 31.1,+06 37,90,45,45
freq,1,1,1,1,1,5,5.0,1,2,68,32,6


In [41]:
# Remove rows where 'Type' is NaN
df = df.dropna(subset=['PSA'])

In [42]:
main_df = df
main_df

,Target,Alternate ID,SAO,HIP,WDS,Con,MagC,RA 2000,Dec 2000,Mm,X,PSA
1,HD 21700,BD+27 514,SAO 75964,HIP 16386,STFA 7,Tau,6.8,03 31.1,+27 44,100,45,15
2,Phi Tau,"52 Tau, Alkalbain I",SAO 76558,HIP 20250,SHJ 40,Tau,5,04 20.4,+27 21,90,45,15
3,32 Eri,HR 1212,SAO 130806,HIP 18255,STF 470,Eri,4.5,03 54.3,-02 57,90,140,17
4,Keid,Omicron 2 Eri,SAO 131063,HIP 19849,STF 518,Eri,4.4,04 15.2,-07 40,100,25,17
5,1 Cam,DL Cam,SAO 24672,HIP 21148,STF 550,Cam,5.4,04 32.0,+53 55,90,70,12
6,Beta Cam,10 Cam,SAO 13351,HIP 23522,S 459,Cam,4,05 03.4,+60 27,100,60,11
10,Rho Ori,17 Ori,SAO 112528,HIP 24331,STF 654,Ori,4.6,05 13.3,+02 52,150,100,14
11,Mintaka,"Delta Ori, 34 Ori",SAO 132220,HIP 25930,BU 558 / STFA 14,Ori,2.4,05 32.0,-00 18,90,45,16
12,HR 1887,HD 36960,SAO 132301,HIP 26199,STF 747,Ori,4.3,05 35.0,-06 00,90,45,16
13,Trapezium,"Theta 1 Ori, 41 Ori",SAO 132314,HIP 26220,STF 748,Ori,6.2,05 35.3,-05 23,90,150,16


## Consolidate

In [43]:
print(main_df.columns)
print(supplement_df.columns)

Index(['Target', 'Alternate ID', 'SAO', 'HIP', 'WDS', 'Con', 'MagC', 'RA 2000',
       'Dec 2000', 'Mm', 'X', 'PSA'],
      dtype='object')
Index(['Target', 'Parent', 'WDS', 'Pair', 'PA', 'Sep', 'M1', 'M2', 'Con',
       'Notes', 'Type', 'season'],
      dtype='object')


In [44]:
# Perform an outer join with supplement_df as the left DataFrame
merged_df = pd.merge(supplement_df, main_df, on='Target', how='left', suffixes=('_supp', ''))

# Strip all strings
merged_df = merged_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Print the columns of the merged DataFrame to verify the merge
print(merged_df.columns)

Index(['Target', 'Parent', 'WDS_supp', 'Pair', 'PA', 'Sep', 'M1', 'M2',
       'Con_supp', 'Notes', 'Type', 'season', 'Alternate ID', 'SAO', 'HIP',
       'WDS', 'Con', 'MagC', 'RA 2000', 'Dec 2000', 'Mm', 'X', 'PSA'],
      dtype='object')


/tmp/ipykernel_18596/636880266.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged_df = merged_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [45]:
merged_df.drop(['Con_supp', 'WDS'], axis=1, inplace=True)
# Keep rows where 'Target' column values are not 'Target'
merged_df = merged_df.loc[merged_df['Target'] != 'Target']
merged_df.rename(columns={
    'WDS_supp': 'WDS',
    'Sep': 'SepSec'
}, inplace=True)
merged_df['Type'].replace("binary", "Binary", inplace=True)
merged_df['Type'].replace("optical", "Optical", inplace=True)
merged_df = merged_df[['Target', 'Alternate ID', 'SAO', 'HIP', 'WDS', 'Con', 'RA 2000',
       'Dec 2000', 'Mm', 'X', 'PSA', 'season', 'Parent', 'Pair',
       'PA', 'SepSec', 'MagC', 'M1', 'M2', 'Notes', 'Type']]
merged_df['Type'].unique()
merged_df = merged_df.fillna('')


In [46]:
merged_df.describe()

,PA,SepSec,M1,M2
count,235.000000,235.000000,235.000000,235.000000
mean,177.200000,105.533191,5.603830,8.897447
std,105.938331,152.196732,1.500465,2.261059
min,0.000000,1.100000,2.000000,4.600000
25%,84.000000,15.050000,4.600000,7.100000
50%,175.000000,48.800000,5.700000,8.800000
75%,267.000000,124.000000,6.600000,10.850000
max,358.000000,905.100000,11.600000,12.800000


## format for PiFinder

### first an export of the full dataset, before pruning

In [47]:
merged_df.to_csv('rasc_double_stars_full.csv', sep='\t', index=False)

In [48]:
df = merged_df.drop(['SAO', 'HIP', 'PSA', 'Parent'], axis=1, inplace=False)

### Group clusters

In [49]:
non_empty_target = df['Target'] != ''

# Use cumsum on the boolean series to create group identifiers
df['group'] = non_empty_target.cumsum()
# Group the DataFrame by the 'group' column
grouped = df.groupby('group')

### Iterate over groups and extract consolidated notes

In [50]:
# Create a new DataFrame
out_df = pd.DataFrame(columns=['Target', 'AlternateID', 'WDS', 'Con', 'RA2000', 'Dec2000', 'Mag', 'MaxSepSec', 'Notes'])
type_dict = {
    "Optical": "Opt",
    "Physical": "Phy", 
    "Uncertain": "?",
    "Binary": "Bin"
}
def extract_info(row):
    result = f"""{row['Pair']}({type_dict[row['Type']]}) PA:{row['PA']} Sep:{row['SepSec']}<SECS> M:{row['M1']}/{row['M2']}<NEWLINE>{row['Notes']}"""
    return result.replace('"', '<SECS>')
    
# Iterate over each group
for group_number, group_df in grouped:
    #print(f"Processing group {group_number}, {len(group_df)}")
    # 'group_df' is a DataFrame containing only the rows from the current group

    new_row = {'Notes': ''}
    max_sep = -1
    # Perform your operations on each group member here
    # For example, let's print the first row of each group
    letterset = set()
    for index, row in group_df.iterrows():
        pairs = row['Pair'].split(',')
        for pair in pairs:
            letters = [*pair]
            if letters[1].islower():
                letterset.discard(letters[0])
                letters = [letters[1]]
            letterset.update(letters)
    #print(f"{group_number=}, {letterset=}")
    if len(letterset) != len(group_df)+1:
        print(f"Lengnth of letterset is {len(letterset)} == {len(group_df)+1}, {letterset=}, {group_number=}, {group_df=}")
    nr_stars = len(letterset)
         
    for index, row in group_df.iterrows():
        if row['SepSec'] > max_sep:
            max_sep = row['SepSec']
        if row['Target']:
            new_row = {
            'Target': row['Target'],
            'AlternateID': row['Alternate ID'],
            'WDS': row['WDS'],
            'Con': row['Con'],
            'RA2000': row['RA 2000'], 
            'Dec2000': row['Dec 2000'],
            'Mag': [row['MagC'], row['M1'], row['M2']],
            'Notes': f"{nr_stars} 󰥮{row['Mm']/10:g}cm {row['X']:g}x {row['season']}<NEWLINE>{extract_info(row)}"
            }
        else:
            new_row['Notes'] = new_row['Notes'] + '<NEWLINE>' + extract_info(row)

    # Append the new row to the DataFrame
    new_row['MaxSepSec'] = max_sep
    out_df = pd.concat([out_df,  pd.DataFrame([new_row])], ignore_index=True)
out_df

/tmp/ipykernel_18596/13851462.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([out_df,  pd.DataFrame([new_row])], ignore_index=True)


,Target,AlternateID,WDS,Con,RA2000,Dec2000,Mag,MaxSepSec,Notes
0,HD 21700,BD+27 514,STFA 7,Tau,03 31.1,+27 44,"[6.8, 7.4, 7.8]",44.1,2 󰥮10cm 45x Win1<NEWLINE>AB(Phy) PA:234 Sep:44.1<SECS> M:7.4/7.8<NEWLINE>Wide pair. Nearly equal. Oriented NE to SW. Yellow or orange and blue. Beside double HR 1065.
1,Phi Tau,"52 Tau, Alkalbain I",SHJ 40,Tau,04 20.4,+27 21,"[5.0, 5.1, 7.5]",118.1,"3 󰥮9cm 45x Win1<NEWLINE>AB(Opt) PA:259 Sep:48.7<SECS> M:5.1/7.5<NEWLINE>Widely separated. Intense colours, gold and blue. Unrelated bright orange-red star opposite B.<NEWLINE>AC(?) PA:25 Sep:118.1<SECS> M:5.1/12.3<NEWLINE>Very dim partner, NNE."
2,32 Eri,HR 1212,STF 470,Eri,03 54.3,-02 57,"[4.5, 4.8, 5.9]",164.7,"3 󰥮9cm 140x Win1<NEWLINE>AB(?) PA:349 Sep:6.9<SECS> M:4.8/5.9<NEWLINE>Colourful pair, yellow and pale blue. Tight. May be called HR 1211 as opposed to HR 1212.<NEWLINE>AC(Opt) PA:5 Sep:164.7<SECS> M:4.8/10.5<NEWLINE>Dim star beyond secondary, further north. Colourless."
3,Keid,Omicron 2 Eri,STF 518,Eri,04 15.2,-07 40,"[4.4, 4.4, 9.3]",481.4,"4 󰥮10cm 25x Win1<NEWLINE>AB(Phy) PA:102 Sep:83.7<SECS> M:4.4/9.3<NEWLINE>Very wide. Primary is orange or yellow. B is blue or red? B nearly due east.<NEWLINE>AC(Phy) PA:98 Sep:78.1<SECS> M:4.4/11.2<NEWLINE>Another dim companion. C is very close to B, 9<SECS> apart! C north of B.<NEWLINE>AD(Opt) PA:38 Sep:481.4<SECS> M:4.4/12.6<NEWLINE>Very faint and a long ways away. Looks like a field star. To the NE."
4,1 Cam,DL Cam,STF 550,Cam,04 32.0,+53 55,"[5.4, 5.8, 6.8]",150.9,3 󰥮9cm 70x Win1<NEWLINE>AB(?) PA:309 Sep:10.4<SECS> M:5.8/6.8<NEWLINE>A and B bright. B NW of A. Very tight. Both look blue-white. A showcase pair for many.<NEWLINE>BC(Opt) PA:217 Sep:150.9<SECS> M:6.8/11.4<NEWLINE>C much dimmer and well away to the SW. Looks pale blue?
5,Beta Cam,10 Cam,S 459,Cam,05 03.4,+60 27,"[4.0, 4.1, 7.4]",84.2,3 󰥮10cm 60x Win1<NEWLINE>AB(?) PA:209 Sep:84.2<SECS> M:4.1/7.4<NEWLINE>Wide pair. Yellow and deep blue. B to the SW. Lots of field stars.<NEWLINE>BC(Phy) PA:170 Sep:14.8<SECS> M:7.4/12.4<NEWLINE>Interesting triple. Very dim. Close to B. In a hockey stick shape. C south of B.
6,Rho Ori,17 Ori,STF 654,Ori,05 13.3,+02 52,"[4.6, 4.6, 8.5]",183.4,"3 󰥮15cm 100x Win1<NEWLINE>AB(?) PA:65 Sep:6.4<SECS> M:4.6/8.5<NEWLINE>Fantastic colours. Gold and deep blue. Quite tight. To the NE.<NEWLINE>AC(Opt) PA:157 Sep:183.4<SECS> M:4.6/11.4<NEWLINE>90° to A and B. Very far away, 20 to 25 times!"
7,Mintaka,"Delta Ori, 34 Ori",STFA 14,Ori,05 32.0,-00 18,"[2.4, 2.4, 6.8]",56.2,"2 󰥮9cm 45x Win1<NEWLINE>AC(Opt) PA:4 Sep:56.2<SECS> M:2.4/6.8<NEWLINE>Widely separated, very attractive. Primary is white, a little yellow. Next star medium blue."
8,HR 1887,HD 36960,STF 747,Ori,05 35.0,-06 00,"[4.3, 4.7, 5.5]",68.2,"3 󰥮9cm 45x Win1<NEWLINE>AB(Phy) PA:225 Sep:36.0<SECS> M:4.7/5.5<NEWLINE>Very similar, pale yellow. Easily separated at 145x. Near Iota Orionis and Messier 42.<NEWLINE>AC(Phy) PA:12 Sep:68.2<SECS> M:4.7/9.0<NEWLINE>Faint companion to the N."
9,Trapezium,"Theta 1 Ori, 41 Ori",STF 748,Ori,05 35.3,-05 23,"[6.2, 6.6, 7.5]",21.5,"7 󰥮9cm 150x Win1<NEWLINE>AB(?) PA:32 Sep:8.8<SECS> M:6.6/7.5<NEWLINE>The Trapezium in the middle of M42. Stunning. B is NNE of A. Both white. Some report diff. colours.<NEWLINE>AC(Opt) PA:132 Sep:12.8<SECS> M:6.6/5.1<NEWLINE>A, B, C, and D are all fairly easy. C is SE of A.<NEWLINE>AD(?) PA:97 Sep:21.5<SECS> M:6.6/6.4<NEWLINE>A, B, C, and D are all fairly easy. D is kitty-corner from and east of A.<NEWLINE>AE(?) PA:352 Sep:4.6<SECS> M:6.6/11.1<NEWLINE>E and F get harder. Need good seeing. E is between A and B (but not in-line).<NEWLINE>BF(?) PA:154 Sep:20.5<SECS> M:7.5/11.5<NEWLINE>And higher magnification. F is SE of C.<NEWLINE>CZ(?) PA:338 Sep:6.5<SECS> M:5.1/12.7<NEWLINE>Bonus points!"


In [51]:
out_df.index = out_df.index + 1
out_df = out_df.reset_index().rename(columns={'index': 'sequence'})
out_df.to_csv('rasc_double_stars.csv', sep='\t', index=False)